In [14]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import pandas as pd
import cv2

In [2]:
image_size = 224

In [4]:
base_model = keras.applications.Xception(
    weights='imagenet',
    input_shape=(image_size, image_size, 3),
    include_top=False)

# Create new model on top.
inputs = keras.Input(shape=(image_size, image_size, 3))
x = base_model(inputs, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
outputs = keras.layers.Dense(13)(x)
model = keras.Model(inputs, outputs)
model.summary()


model = tf.keras.models.load_model('models/Xception_94.17.h5')

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
xception (Model)             (None, 7, 7, 2048)        20861480  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 13)                26637     
Total params: 20,888,117
Trainable params: 20,833,589
Non-trainable params: 54,528
_________________________________________________________________


In [6]:
model.save('xception')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: xception/assets


In [7]:
# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model("xception") # path to the SavedModel directory
tflite_model = converter.convert()

# Save the model.
with open('xception.tflite', 'wb') as f:
    f.write(tflite_model)

In [11]:
# validation data
val_x = pd.read_csv('val.zip')
val_y = pd.read_csv('val_labels.zip')

x_val = np.array(val_x)[:,1:].reshape((val_x.shape[0],image_size,image_size,3))[...,::-1].astype('uint8')
y_val = np.array(val_y.iloc[:,1])

In [15]:
for i in range(5):
    index = np.random.randint(360)
    print(f"====TF Model result{index}====")
    print(model.predict(np.expand_dims(x_val[index],axis=0)))
    print(y_val[index])

    interpreter = tf.lite.Interpreter(model_path="InceptionResNetV2.tflite")
    interpreter.allocate_tensors()
    # Get input and output tensors.
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    # Test the model on random input data.
    input_shape = input_details[0]['shape']

    img = cv2.resize(x_val[index], (image_size,image_size)).astype('float32')

    input_data = np.expand_dims(img, axis = 0)
    interpreter.set_tensor(input_details[0]['index'], input_data)

    interpreter.invoke()

    # The function `get_tensor()` returns a copy of the tensor data.
    # Use `tensor()` in order to get a pointer to the tensor.
    output_data = interpreter.get_tensor(output_details[0]['index'])
    print("====TFLite result====")
    print(output_data)
    print(np.argmax(output_data))

====TF Model result346====
[[-17.901442  -23.404106  -32.070164  -14.11898   -21.601423   -1.850947
   24.419962   -4.4148817  -2.6606011   9.388392  -36.647743  -12.99203
  -11.208064 ]]
6
====TFLite result====
[[-4.1809664   2.3012254  -2.916342    5.5937896  -1.2589072   5.38928
   9.906274    8.962482    6.942103    3.748101    0.47662482 10.412812
  -0.5057837 ]]
11
====TF Model result65====
[[  3.914014   -5.747334   13.94556    -4.4586163  -8.754163  -10.760125
    3.21997     7.268593   -1.9283341   5.1792164   3.2062905   5.978146
   -4.1129036]]
2
====TFLite result====
[[ 0.12046222 -0.25927347 10.88987    -0.72205186 -2.940135   -6.352611
   5.2650185   5.7163672   0.27584827  3.8899827   2.6346333   4.300824
  -2.6199255 ]]
2
====TF Model result76====
[[  0.58516854   0.23161644  18.535362    -5.8178697  -10.962278
  -12.415214     8.200231    10.053365    -5.309121     4.216158
   -1.4761102    3.3084779   -5.2805204 ]]
2
====TFLite result====
[[-1.40600335e-02 -2.82122207

In [17]:
image_size = 331
base_model = keras.applications.NASNetLarge(
    weights='imagenet',
    input_shape=(image_size, image_size, 3),
    include_top=False)
inputs = keras.Input(shape=(image_size, image_size, 3))
x = base_model(inputs, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
outputs = keras.layers.Dense(13)(x)
model = keras.Model(inputs, outputs)
model.summary()


model = tf.keras.models.load_model('models/NASNetLarge.9306.h5')
model.save('NASNetLarge')
# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model("NASNetLarge") # path to the SavedModel directory
tflite_model = converter.convert()

# Save the model.
with open('NASNetLarge.tflite', 'wb') as f:
    f.write(tflite_model)
    
    
for i in range(5):
    index = np.random.randint(360)
    print(f"====TF Model result{index}====")
    print(model.predict(np.expand_dims(x_val[index],axis=0)))
    print(y_val[index])

    interpreter = tf.lite.Interpreter(model_path="InceptionResNetV2.tflite")
    interpreter.allocate_tensors()
    # Get input and output tensors.
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    # Test the model on random input data.
    input_shape = input_details[0]['shape']

    img = cv2.resize(x_val[index], (image_size,image_size)).astype('float32')

    input_data = np.expand_dims(img, axis = 0)
    interpreter.set_tensor(input_details[0]['index'], input_data)

    interpreter.invoke()

    # The function `get_tensor()` returns a copy of the tensor data.
    # Use `tensor()` in order to get a pointer to the tensor.
    output_data = interpreter.get_tensor(output_details[0]['index'])
    print("====TFLite result====")
    print(output_data)
    print(np.argmax(output_data))

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 331, 331, 3)]     0         
_________________________________________________________________
NASNet (Model)               (None, 11, 11, 4032)      84916818  
_________________________________________________________________
global_average_pooling2d_2 ( (None, 4032)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 13)                52429     
Total params: 84,969,247
Trainable params: 84,772,579
Non-trainable params: 196,668
_________________________________________________________________
INFO:tensorflow:Assets written to: NASNetLarge/assets


INFO:tensorflow:Assets written to: NASNetLarge/assets


====TF Model result324====


[[ -8.5282135 -19.363518  -16.52757   -11.873962   -5.7922344  -8.415912
   23.99141    -2.5310426   5.828314    1.085365  -12.8724375  -3.8069303
  -15.421123 ]]
6
====TFLite result====
[[-1.3825101  1.6070299 -0.1115901 -1.5724983  1.0438304  5.835176
  23.378407   7.1895914 14.445209   8.269168   5.057511   2.975391
  -2.8058362]]
6
====TF Model result245====
[[ -3.629729    -0.9774473   -7.341594    -0.22208425 -11.239543
   -9.283462     0.11140212   1.7786671   -4.116989    -3.5408378
    9.306289    -1.1097876  -10.166046  ]]
10
====TFLite result====
[[ 0.7998347 10.453156  -1.2456601 -5.1298923 -1.3739543 -6.4387946
  11.405606  10.661482   2.9767356  4.238061  21.551626   6.246598
  -7.3708577]]
10
====TF Model result215====
[[ -1.3384378  -1.2458938  -6.145743    2.2396846 -10.531468   -9.579082
   -1.8510113   1.5018973  -2.8273437  -4.362086   10.495873    0.3627963
   -9.966937 ]]
10
====TFLite result====
[[ 2.999232    6.6426644  -1.3552171   0.898814   -1.710811   -4.716